# TABLAS RIPS 

## Carga de los datos

Este jupyter notebook esta generando las tablas sobre el año 2021 de la fuente de datos RIPS

In [1]:
from pyspark.sql import SparkSession

# Crear una sesión de Spark
spark = SparkSession.builder.appName("GeneraciónTablasRips_Todoslosdatos2021").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/14 22:18:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# Leer los archivos Parquet de un año 
df1_parquet = spark.read.parquet("/rawdata/rips/rips_parquet_2021")

df=df1_parquet

In [3]:
# Contar el número de registros en el DataFrame unido
total_registros = df.count()

# Listar las columnas del DataFrame unido
columnas = df.columns

# Mostrar resultados
print(f"Total de registros en el DataFrame unido: {total_registros}")
print(f"Columnas del DataFrame unido: {columnas}")

Total de registros en el DataFrame unido: 406833792
Columnas del DataFrame unido: ['personaid', 'tipoeventoripsdesc', 'codigo', 'dxprincipal', 'dxegreso', 'finalidadprocedimientoscd', 'finalidadconsultacd', 'tipousuariocd', 'causaexternacd', 'prestador', 'estadosalidacd', 'tipodiagnosticoprincipalcd', 'codigoprocedimiento', 'municipiocd', 'fechaid', 'edad', 'sexodesc', 'costoconsulta', 'costoprocedimiento', 'netoapagarconsulta', 'numerodiasestancia', '__null_dask_index__']


In [4]:
# Contar el número de registros - da satisfactoriamente la cantidad de registros de toda la fuente RIPS 4’262.860.903 registros 
record_count = df.count()

# Mostrar el número de registros
print(f"El número de registros en el archivo es: {record_count}")

El número de registros en el archivo es: 406833792


In [5]:
# Contar el número total de columnas en el DataFrame ( no se tiene en cuenta neto a pagar)
total_columns = len(df.columns)

# Imprimir el número total de columnas
print("Número total de columnas en el DataFrame:", total_columns)

Número total de columnas en el DataFrame: 22


In [6]:
# Mostrar el esquema (columnas y tipos de datos)
df.printSchema()

# Listar las columnas
print(df.columns)

root
 |-- personaid: string (nullable = true)
 |-- tipoeventoripsdesc: string (nullable = true)
 |-- codigo: string (nullable = true)
 |-- dxprincipal: string (nullable = true)
 |-- dxegreso: string (nullable = true)
 |-- finalidadprocedimientoscd: string (nullable = true)
 |-- finalidadconsultacd: string (nullable = true)
 |-- tipousuariocd: string (nullable = true)
 |-- causaexternacd: string (nullable = true)
 |-- prestador: string (nullable = true)
 |-- estadosalidacd: string (nullable = true)
 |-- tipodiagnosticoprincipalcd: string (nullable = true)
 |-- codigoprocedimiento: string (nullable = true)
 |-- municipiocd: string (nullable = true)
 |-- fechaid: string (nullable = true)
 |-- edad: string (nullable = true)
 |-- sexodesc: string (nullable = true)
 |-- costoconsulta: string (nullable = true)
 |-- costoprocedimiento: string (nullable = true)
 |-- netoapagarconsulta: string (nullable = true)
 |-- numerodiasestancia: string (nullable = true)
 |-- __null_dask_index__: long (nul

# Empieza el proceso de generación de los datos agrupados por atenciones

In [6]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Crear una sesión de Spark
spark = SparkSession.builder.appName("DataFrame agrupado").getOrCreate()


# Cargar el archivo Parquet en un DataFrame
#df = df1

# Seleccionar solo las columnas deseadas (sin 'personaid_count' ya que no existe aún)
columns_to_keep = [
    'personaid', 'codigoprocedimiento', 'tipoeventoripsdesc', 'fechaid',
    'sexodesc', 'tipodiagnosticoprincipalcd', 'edad', 'dxegreso', 'dxprincipal', 'municipiocd'
]
df_filtered = df.select(*columns_to_keep)

# Agregar una nueva columna con el recuento distintivo de 'personaid'
window_spec = Window.partitionBy('personaid')
df_filtered = df_filtered.withColumn('personaid_count', F.count('personaid').over(window_spec))

# Mostrar las primeras filas del DataFrame filtrado para verificar
df_filtered.show()

# Mostrar los nombres de las columnas filtradas para verificar
#print("Nombres de las columnas filtradas:")
#print(df_filtered.columns)

# Agrupar por la columna 'personaid'
grouped_df = df_filtered.groupBy('personaid').agg(
    F.first('codigoprocedimiento').alias('codigoprocedimiento'),
    F.first('tipoeventoripsdesc').alias('tipoeventoripsdesc'),
    F.first('fechaid').alias('fechaid'),
    F.first('sexodesc').alias('sexodesc'),
    F.first('tipodiagnosticoprincipalcd').alias('tipodiagnosticoprincipalcd'),
    F.first('edad').alias('edad'),
    F.first('dxegreso').alias('dxegreso'),
    F.first('dxprincipal').alias('dxprincipal'),
    F.first('municipiocd').alias('municipiocd'),
    F.first('personaid_count').alias('personaid_count')
)

# Mostrar las primeras filas del DataFrame agrupado para verificar
grouped_df.show()


24/08/14 22:19:49 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+---------+-------------------+--------------------+--------+---------+--------------------------+----+--------+-----------+-----------+---------------+
|personaid|codigoprocedimiento|  tipoeventoripsdesc| fechaid| sexodesc|tipodiagnosticoprincipalcd|edad|dxegreso|dxprincipal|municipiocd|personaid_count|
+---------+-------------------+--------------------+--------+---------+--------------------------+----+--------+-----------+-----------+---------------+
|100000046|             890207|           CONSULTAS|20210731| FEMENINO|                         1|  27|       0|       Z010|       null|             19|
|100000046|             890203|           CONSULTAS|20211101| FEMENINO|                         1|  27|       0|       K088|      11001|             19|
|100000046|             906340|PROCEDIMIENTOS DE...|20210618| FEMENINO|                      null|  28|       0|          1|      25430|             19|
|100000046|             890301|           CONSULTAS|20210317| FEMENINO|           

+---------+-------------------+--------------------+--------+---------+--------------------------+----+--------+-----------+-----------+---------------+
|personaid|codigoprocedimiento|  tipoeventoripsdesc| fechaid| sexodesc|tipodiagnosticoprincipalcd|edad|dxegreso|dxprincipal|municipiocd|personaid_count|
+---------+-------------------+--------------------+--------+---------+--------------------------+----+--------+-----------+-----------+---------------+
|100000046|             890203|           CONSULTAS|20211001| FEMENINO|                         1|  27|       0|       K088|       null|             19|
|100001340|             890284|           CONSULTAS|20210623|MASCULINO|                         1|  27|       0|       F321|      05266|              4|
|100002120|             952001|PROCEDIMIENTOS DE...|20211214|MASCULINO|                      null|  32|       0|          1|      15542|              1|
|100002546|             881201|PROCEDIMIENTOS DE...|20210114| FEMENINO|           

In [7]:
# Contar el número de registros - me da satisfactoriamente la cantidad de regustros en cada año da 73'357.494 llego hasta stage 18
#llega a 30'434.249 con 2022
record_count = grouped_df.count()


# Mostrar el número de registros
print(f"El número de registros en el dataframe es: {record_count}")

El número de registros en el dataframe es: 32494184


Se generan un total de 73'357.494 registros (todos) para el año 2022 son 30'434.249  registros

In [8]:
 df_final =grouped_df


In [10]:
# Contar el número de registros 
record_count = df_final.count()

# Mostrar el número de registros
print(f"El número de registros en el archivo Parquet es: {record_count}")

El número de registros en el archivo Parquet es: 32494184


In [11]:
# Mostrar las primeras filas del DataFrame agrupado para verificar
df_final.show()

+---------+-------------------+--------------------+--------+---------+--------------------------+----+--------+-----------+-----------+---------------+
|personaid|codigoprocedimiento|  tipoeventoripsdesc| fechaid| sexodesc|tipodiagnosticoprincipalcd|edad|dxegreso|dxprincipal|municipiocd|personaid_count|
+---------+-------------------+--------------------+--------+---------+--------------------------+----+--------+-----------+-----------+---------------+
|100000046|             903815|PROCEDIMIENTOS DE...|20210305| FEMENINO|                      null|  26|       0|          1|       null|             19|
|100001340|             890284|           CONSULTAS|20210826|MASCULINO|                         1|  27|       0|       F321|      05266|              4|
|100002120|             952001|PROCEDIMIENTOS DE...|20211214|MASCULINO|                      null|  32|       0|          1|      15542|              1|
|100002546|             890206|           CONSULTAS|20210112| FEMENINO|           

In [12]:
#se escribe el parquet - No lo hago por que seria todos los datos por atenciones, mejor solo de cada tabla
#df_final.write.parquet("hdfs:/analyticsdata/todoRips.parquet") 

# Empieza el proceso de generación de tablas

## 1. Agrupado por año y mes

In [10]:
# Cargar el archivo Parquet en un DataFrame
df2 = df_final

In [14]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Crear una sesión de Spark
spark = SparkSession.builder.appName("DataFrameTablas ").getOrCreate()


# Cargar el archivo Parquet en un DataFrame
df2 = df_final




24/08/12 23:27:45 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [11]:
# Contar el número de registros - me da satisfactoriamente la cantidad de regustros en cada año 
record_count = df2.count()

# Mostrar el número de registros
print(f"El número de registros en el archivo Parquet es: {record_count}")

El número de registros en el archivo Parquet es: 32494184


In [16]:
# Mostrar el esquema (columnas y tipos de datos)
df2.printSchema()

# Listar las columnas
print(df2.columns)

root
 |-- personaid: string (nullable = true)
 |-- codigoprocedimiento: string (nullable = true)
 |-- tipoeventoripsdesc: string (nullable = true)
 |-- fechaid: string (nullable = true)
 |-- sexodesc: string (nullable = true)
 |-- tipodiagnosticoprincipalcd: string (nullable = true)
 |-- edad: string (nullable = true)
 |-- dxegreso: string (nullable = true)
 |-- dxprincipal: string (nullable = true)
 |-- municipiocd: string (nullable = true)
 |-- personaid_count: long (nullable = true)

['personaid', 'codigoprocedimiento', 'tipoeventoripsdesc', 'fechaid', 'sexodesc', 'tipodiagnosticoprincipalcd', 'edad', 'dxegreso', 'dxprincipal', 'municipiocd', 'personaid_count']


# resultado final agrupado año y mes, según el numero de personas y atenciones

In [18]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'personaid', 'codigoprocedimiento', 'tipoeventoripsdesc', 'fechaid',
    'sexodesc', 'tipodiagnosticoprincipalcd', 'edad', 'dxegreso', 'dxprincipal', 'municipiocd', 'personaid_count'
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'fechaid' a tipo fecha si es necesario
df_filtered = df_filtered.withColumn('fechaid', F.to_date('fechaid', 'yyyyMMdd'))

# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('fechaid'))
df_filtered = df_filtered.withColumn('Mes', F.month('fechaid'))

# Agrupar por año y mes
result_df = df_filtered.groupBy('Año', 'Mes').agg(
    F.countDistinct('personaid').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Filtrar para mostrar los resultados de enero para comprobar
#result_january = result_df.filter((F.col('Año') == 2022) & (F.col('Mes') == 1))

# Mostrar los resultados para enero
#result_january.show()

# Mostrar las primeras filas del DataFrame completo para verificar
result_df.show()

# Imprimir los nombres de las columnas agrupadas para verificar
#print("Nombres de las columnas agrupadas:")
#print(result_df.columns)

24/08/12 23:32:59 WARN TaskSetManager: Lost task 37.0 in stage 36.0 (TID 1726) (10.34.1.49 executor 1): java.io.IOException: unexpected exception type
	at java.io.ObjectStreamClass.throwMiscException(ObjectStreamClass.java:1750)
	at java.io.ObjectStreamClass.invokeReadResolve(ObjectStreamClass.java:1280)
	at java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:2221)
	at java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1668)
	at java.io.ObjectInputStream.defaultReadFields(ObjectInputStream.java:2430)
	at java.io.ObjectInputStream.readSerialData(ObjectInputStream.java:2354)
	at java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:2212)
	at java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1668)
	at java.io.ObjectInputStream.readArray(ObjectInputStream.java:2118)
	at java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1656)
	at java.io.ObjectInputStream.defaultReadFields(ObjectInputStream.java:2430)
	at java.io.ObjectInputStrea

+----+----+------------------+--------------------+
| Año| Mes|Número_de_Personas|Número_de_Atenciones|
+----+----+------------------+--------------------+
|2021|   3|           2267553|            18766382|
|2021|  11|           2687291|            31152014|
|2021|   2|           2036350|            14960822|
|2021|   7|           3482882|            52925160|
|2021|   5|           2421314|            25209590|
|2021|   1|           1171941|             9507549|
|2021|  12|           2741574|            36575220|
|2021|   8|           3467409|            52586249|
|2021|   9|           3748318|            57630256|
|2021|   6|           3450735|            47918268|
|2021|  10|           2842535|            39670851|
|2021|   4|           2176280|            19931429|
|null|null|                 2|                   2|
+----+----+------------------+--------------------+



In [19]:
result_df.show()

+----+----+------------------+--------------------+
| Año| Mes|Número_de_Personas|Número_de_Atenciones|
+----+----+------------------+--------------------+
|2021|   3|           2247344|            18499308|
|2021|  11|           2695872|            31151131|
|2021|   2|           2044908|            14948745|
|2021|   7|           3501982|            53036566|
|2021|   5|           2400304|            24842570|
|2021|   1|           1175290|             9543987|
|2021|  12|           2832276|            38682657|
|2021|   8|           3465880|            52133352|
|2021|   9|           3734553|            57605925|
|2021|   6|           3416121|            47485418|
|2021|  10|           2817672|            39111292|
|2021|   4|           2161980|            19792839|
|null|null|                 2|                   2|
+----+----+------------------+--------------------+



Modificar el nombre del archivo segun corresponda el año

In [20]:
#se escribe el parquet tabla 1
result_df.write.parquet("hdfs:/analyticsdata/1.Tabla_anio_mes_atenciones_2021.parquet") 

# 2. Agrupado por año, mes y sexo

In [21]:

# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'personaid', 'codigoprocedimiento', 'tipoeventoripsdesc', 'fechaid',
    'sexodesc', 'tipodiagnosticoprincipalcd', 'edad', 'dxegreso', 'dxprincipal', 'municipiocd', 'personaid_count'
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'fechaid' a tipo fecha si es necesario
df_filtered = df_filtered.withColumn('fechaid', F.to_date('fechaid', 'yyyyMMdd'))

# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('fechaid'))
df_filtered = df_filtered.withColumn('Mes', F.month('fechaid'))

# Agrupar por año, mes y sexodesc
result_df = df_filtered.groupBy('Año', 'Mes', 'sexodesc').agg(
    F.countDistinct('personaid').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)


+----+----+-----------+------------------+--------------------+
| Año| Mes|   sexodesc|Número_de_Personas|Número_de_Atenciones|
+----+----+-----------+------------------+--------------------+
|2021|   3|   FEMENINO|           1265355|            11231956|
|2021|   3|       null|              2318|               36396|
|2021|  12|NO DEFINIDO|              9194|               61863|
|2021|   5|  MASCULINO|           1095451|             9950472|
|2021|  11|       null|              4640|               67634|
|2021|  12|   FEMENINO|           1550401|            22891776|
|2021|  10|   FEMENINO|           1627924|            24773170|
|2021|   4|   FEMENINO|           1190153|            11920990|
|2021|   2|       null|              1866|               24763|
|2021|   1|  MASCULINO|            541126|             3999230|
|2021|   6|  MASCULINO|           1496615|            18034051|
|2021|   9|NO DEFINIDO|              9497|               69136|
|2021|  11|  MASCULINO|           114144

In [22]:
#se escribe el parquet tabla 2
result_df.write.parquet("hdfs:/analyticsdata/2.Tabla_anio_mes_sexo_2021.parquet") 

# 3. Agrupado por año, mes y municipio

In [23]:

# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'personaid', 'codigoprocedimiento', 'tipoeventoripsdesc', 'fechaid',
    'sexodesc', 'tipodiagnosticoprincipalcd', 'edad', 'dxegreso', 'dxprincipal', 'municipiocd', 'personaid_count'
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'fechaid' a tipo fecha si es necesario
df_filtered = df_filtered.withColumn('fechaid', F.to_date('fechaid', 'yyyyMMdd'))

# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('fechaid'))
df_filtered = df_filtered.withColumn('Mes', F.month('fechaid'))

# Agrupar por año, mes y municipioid
result_df = df_filtered.groupBy('Año', 'Mes', 'municipiocd').agg(
    F.countDistinct('personaid').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)


+----+---+-----------+------------------+--------------------+
| Año|Mes|municipiocd|Número_de_Personas|Número_de_Atenciones|
+----+---+-----------+------------------+--------------------+
|2021|  7|      19355|               706|                8036|
|2021|  3|      44090|              1435|                7991|
|2021| 12|      20238|              2069|               29588|
|2021|  2|      86885|              1171|               12740|
|2021|  4|      41359|              1716|               12265|
|2021|  1|      19824|               168|                1380|
|2021|  6|      19573|              4344|               64056|
|2021|  7|      27150|               244|                2612|
|2021|  4|      70265|              1017|                7288|
|2021|  6|      23574|               757|                8522|
|2021|  5|      44420|               271|                2691|
|2021|  3|      05861|               174|                 835|
|2021|  9|      68245|               200|              

In [24]:
#se escribe el parquet tabla 3
result_df.write.parquet("hdfs:/analyticsdata/3.Tabla_anio_mes_municipio_2021.parquet") 

# 4. Agrupado por año, mes y tipoeventoripsdesc

In [25]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'personaid', 'codigoprocedimiento', 'tipoeventoripsdesc', 'fechaid',
    'sexodesc', 'tipodiagnosticoprincipalcd', 'edad', 'dxegreso', 'dxprincipal', 'municipiocd', 'personaid_count'
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'fechaid' a tipo fecha si es necesario
df_filtered = df_filtered.withColumn('fechaid', F.to_date('fechaid', 'yyyyMMdd'))

# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('fechaid'))
df_filtered = df_filtered.withColumn('Mes', F.month('fechaid'))

# Agrupar por año, mes y tipoeventoripsdesc
result_df = df_filtered.groupBy('Año', 'Mes', 'tipoeventoripsdesc').agg(
    F.countDistinct('personaid').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)

+----+----+--------------------+------------------+--------------------+
| Año| Mes|  tipoeventoripsdesc|Número_de_Personas|Número_de_Atenciones|
+----+----+--------------------+------------------+--------------------+
|2021|   1|PROCEDIMIENTOS DE...|            496098|             3699544|
|2021|  11|PROCEDIMIENTOS DE...|            954424|            12218237|
|2021|   1|           CONSULTAS|            618321|             5180151|
|2021|   4|PROCEDIMIENTOS DE...|            748304|             6877939|
|2021|   9|           CONSULTAS|           2222993|            33466015|
|2021|   5|           URGENCIAS|             57964|              692952|
|2021|   3|PROCEDIMIENTOS DE...|            727081|             6241197|
|2021|   4|           URGENCIAS|             45084|              492889|
|2021|  12|           URGENCIAS|             42484|              649559|
|2021|   2|PROCEDIMIENTOS DE...|            699808|             5002813|
|2021|   5|           CONSULTAS|           1487559|

In [26]:
#se escribe el parquet tabla 4
result_df.write.parquet("hdfs:/analyticsdata/4.Tabla_anio_mes_tipoeventoripsdesc_2021.parquet") 

# 5. Agrupado por año, mes y tipodiagnosticoprincipalcd

In [27]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'personaid', 'codigoprocedimiento', 'tipoeventoripsdesc', 'fechaid',
    'sexodesc', 'tipodiagnosticoprincipalcd', 'edad', 'dxegreso', 'dxprincipal', 'municipiocd', 'personaid_count'
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'fechaid' a tipo fecha si es necesario
df_filtered = df_filtered.withColumn('fechaid', F.to_date('fechaid', 'yyyyMMdd'))

# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('fechaid'))
df_filtered = df_filtered.withColumn('Mes', F.month('fechaid'))

# Agrupar por año, mes y tipodiagnosticoprincipalcd
result_df = df_filtered.groupBy('Año', 'Mes', 'tipodiagnosticoprincipalcd').agg(
    F.countDistinct('personaid').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)

+----+----+--------------------------+------------------+--------------------+
| Año| Mes|tipodiagnosticoprincipalcd|Número_de_Personas|Número_de_Atenciones|
+----+----+--------------------------+------------------+--------------------+
|2021|   3|                         1|            933470|             7452061|
|2021|   3|                      null|            895896|             7830859|
|2021|   2|                         3|            148555|             1186613|
|2021|  10|                         3|            172608|             3634134|
|2021|   7|                         2|            425890|             6614845|
|2021|   3|                         3|            173826|             1555525|
|2021|  11|                      null|           1026665|            13172951|
|2021|  11|                         2|            298680|             3001399|
|2021|  10|                         2|            296627|             4134483|
|2021|   7|                         3|            30

In [28]:
#se escribe el parquet tabla 5
result_df.write.parquet("hdfs:/analyticsdata/5.Tabla_anio_mes_tipodiagnosticoprincipalcd_2021.parquet") 

# 6. Agrupado por año, mes y dxprincipal

In [29]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'personaid', 'codigoprocedimiento', 'tipoeventoripsdesc', 'fechaid',
    'sexodesc', 'tipodiagnosticoprincipalcd', 'edad', 'dxegreso', 'dxprincipal', 'municipiocd', 'personaid_count'
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'fechaid' a tipo fecha si es necesario
df_filtered = df_filtered.withColumn('fechaid', F.to_date('fechaid', 'yyyyMMdd'))

# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('fechaid'))
df_filtered = df_filtered.withColumn('Mes', F.month('fechaid'))

# Agrupar por año, mes y dxprincipal
result_df = df_filtered.groupBy('Año', 'Mes', 'dxprincipal').agg(
    F.countDistinct('personaid').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)

+----+---+-----------+------------------+--------------------+
| Año|Mes|dxprincipal|Número_de_Personas|Número_de_Atenciones|
+----+---+-----------+------------------+--------------------+
|2021|  2|       S908|                78|                 393|
|2021| 12|       Z008|             16222|              276873|
|2021|  2|       F819|               323|                2465|
|2021| 12|       Z278|              2907|               20232|
|2021| 12|       N644|              1626|               15314|
|2021|  6|       E119|             23206|              543798|
|2021|  4|       O479|               760|               19564|
|2021|  7|       B07X|              2573|               25428|
|2021| 10|       O926|               121|                1572|
|2021| 12|       N942|               124|                1563|
|2021|  4|       B977|               344|                2982|
|2021|  8|       F688|                91|                1073|
|2021| 10|       F840|               520|              

In [30]:
#se escribe el parquet tabla 6
result_df.write.parquet("hdfs:/analyticsdata/6.Tabla_anio_mes_dxprincipal_2021.parquet") 

# 7. Agrupado por año, mes y codigoprocedimiento

In [31]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'personaid', 'codigoprocedimiento', 'tipoeventoripsdesc', 'fechaid',
    'sexodesc', 'tipodiagnosticoprincipalcd', 'edad', 'dxegreso', 'dxprincipal', 'municipiocd', 'personaid_count'
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'fechaid' a tipo fecha si es necesario
df_filtered = df_filtered.withColumn('fechaid', F.to_date('fechaid', 'yyyyMMdd'))

# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('fechaid'))
df_filtered = df_filtered.withColumn('Mes', F.month('fechaid'))

# Agrupar por año, mes y codigoprocedimiento
result_df = df_filtered.groupBy('Año', 'Mes', 'codigoprocedimiento').agg(
    F.countDistinct('personaid').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)

+----+---+-------------------+------------------+--------------------+
| Año|Mes|codigoprocedimiento|Número_de_Personas|Número_de_Atenciones|
+----+---+-------------------+------------------+--------------------+
|2021|  9|             903841|             41811|              884156|
|2021| 12|             240300|               341|                2324|
|2021| 10|             881202|              2477|               41298|
|2021|  2|             890208|             10951|              100181|
|2021|  7|             890454|               307|               37755|
|2021|  5|             873420|              2145|               17463|
|2021|  6|             871050|               113|                1349|
|2021|  8|             906423|                27|                1250|
|2021|  1|             232200|               788|                5382|
|2021|  8|             879122|               197|                1967|
|2021| 11|             896101|               259|                4907|
|2021|

In [32]:
#se escribe el parquet tabla 7
result_df.write.parquet("hdfs:/analyticsdata/7.Tabla_anio_mes_codigoprocedimiento_2021.parquet") 

# 8. Agrupado por año, mes, municipio y sexo

In [33]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'personaid', 'codigoprocedimiento', 'tipoeventoripsdesc', 'fechaid',
    'sexodesc', 'tipodiagnosticoprincipalcd', 'edad', 'dxegreso', 'dxprincipal', 'municipiocd', 'personaid_count'
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'fechaid' a tipo fecha si es necesario
df_filtered = df_filtered.withColumn('fechaid', F.to_date('fechaid', 'yyyyMMdd'))

# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('fechaid'))
df_filtered = df_filtered.withColumn('Mes', F.month('fechaid'))

# Agrupar por año, mes y codigoprocedimiento
result_df = df_filtered.groupBy('Año', 'Mes', 'municipiocd','sexodesc').agg(
    F.countDistinct('personaid').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)

+----+---+-----------+-----------+------------------+--------------------+
| Año|Mes|municipiocd|   sexodesc|Número_de_Personas|Número_de_Atenciones|
+----+---+-----------+-----------+------------------+--------------------+
|2021| 11|      05615|   FEMENINO|              6198|               67266|
|2021|  7|      18001|   FEMENINO|              6994|              159086|
|2021|  1|      44279|   FEMENINO|               743|                6541|
|2021| 10|      68276|   FEMENINO|              7739|              112632|
|2021| 10|      70742|   FEMENINO|               903|               13810|
|2021|  9|      05129|   FEMENINO|              4615|               58179|
|2021|  9|      25183|   FEMENINO|               432|                8126|
|2021|  9|      73547|   FEMENINO|               170|                2641|
|2021|  8|      70742|   FEMENINO|               995|               16378|
|2021|  6|      13222|  MASCULINO|               378|                3945|
|2021|  4|      20175|  M

In [34]:
#se escribe el parquet tabla 8
result_df.write.parquet("hdfs:/analyticsdata/8.Tabla_anio_mes_municipio_sexo_2021.parquet") 

# 9. Agrupado por año, mes, tipoeventorips y sexo

In [35]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'personaid', 'codigoprocedimiento', 'tipoeventoripsdesc', 'fechaid',
    'sexodesc', 'tipodiagnosticoprincipalcd', 'edad', 'dxegreso', 'dxprincipal', 'municipiocd', 'personaid_count'
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'fechaid' a tipo fecha si es necesario
df_filtered = df_filtered.withColumn('fechaid', F.to_date('fechaid', 'yyyyMMdd'))

# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('fechaid'))
df_filtered = df_filtered.withColumn('Mes', F.month('fechaid'))

# Agrupar por año, mes y codigoprocedimiento
result_df = df_filtered.groupBy('Año', 'Mes', 'tipoeventoripsdesc','sexodesc').agg(
    F.countDistinct('personaid').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)

+----+----+--------------------+-----------+------------------+--------------------+
| Año| Mes|  tipoeventoripsdesc|   sexodesc|Número_de_Personas|Número_de_Atenciones|
+----+----+--------------------+-----------+------------------+--------------------+
|2021|   1|           URGENCIAS|  MASCULINO|             17468|              139876|
|2021|   1|PROCEDIMIENTOS DE...|       null|               411|                8777|
|2021|  11|           CONSULTAS|  MASCULINO|            714060|             6640377|
|2021|   2|           URGENCIAS|   FEMENINO|             19189|              180322|
|2021|   8|PROCEDIMIENTOS DE...|NO DEFINIDO|              2999|               23058|
|2021|   8|           CONSULTAS|   FEMENINO|           1207083|            20591861|
|2021|   8|           URGENCIAS|  MASCULINO|             31315|              348441|
|2021|  12|           CONSULTAS|NO DEFINIDO|              5032|               30391|
|2021|  11|   HOSPITALIZACIONES|NO DEFINIDO|                41|  

In [36]:
#se escribe el parquet tabla 9
result_df.write.parquet("hdfs:/analyticsdata/9.Tabla_anio_mes_tipoeventoripsdesc_sexo_2021.parquet") 

se comprueba que el archivo quedo guardado correctamente al leerlo 

In [38]:
# Leer el archivo .parquet y se guarda en un DataFrame
df_leer = spark.read.parquet("hdfs:/analyticsdata/9.Tabla_anio_mes_tipoeventoripsdesc_sexo_2021.parquet")

# Mostrar el contenido del DataFrame
df_leer.show()

+----+---+--------------------+-----------+------------------+--------------------+
| Año|Mes|  tipoeventoripsdesc|   sexodesc|Número_de_Personas|Número_de_Atenciones|
+----+---+--------------------+-----------+------------------+--------------------+
|2021|  6|   HOSPITALIZACIONES|  MASCULINO|              5911|              179629|
|2021|  6|           URGENCIAS|NO DEFINIDO|                79|                 454|
|2021|  3|           URGENCIAS|  MASCULINO|             25473|              187571|
|2021|  7|PROCEDIMIENTOS DE...|NO DEFINIDO|              2838|               18741|
|2021|  3|           CONSULTAS|NO DEFINIDO|              1960|                9746|
|2021|  5|   HOSPITALIZACIONES|   FEMENINO|              6817|              201869|
|2021|  5|           CONSULTAS|       null|              1724|               25268|
|2021|  8|           CONSULTAS|NO DEFINIDO|              3831|               31402|
|2021| 10|PROCEDIMIENTOS DE...|       null|              1469|              

# 10. Agrupado por año, mes, edad

In [12]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'personaid', 'codigoprocedimiento', 'tipoeventoripsdesc', 'fechaid',
    'sexodesc', 'tipodiagnosticoprincipalcd', 'edad', 'dxegreso', 'dxprincipal', 'municipiocd', 'personaid_count'
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'fechaid' a tipo fecha si es necesario
df_filtered = df_filtered.withColumn('fechaid', F.to_date('fechaid', 'yyyyMMdd'))

# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('fechaid'))
df_filtered = df_filtered.withColumn('Mes', F.month('fechaid'))

# Agrupar por año, mes y codigoprocedimiento
result_df = df_filtered.groupBy('Año', 'Mes', 'edad').agg(
    F.countDistinct('personaid').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)

+----+---+----+------------------+--------------------+
| Año|Mes|edad|Número_de_Personas|Número_de_Atenciones|
+----+---+----+------------------+--------------------+
|2021|  4|  28|             36920|              295221|
|2021|  5|  60|             25224|              354052|
|2021| 10|  68|             22177|              497302|
|2021| 10|  86|              5328|              159282|
|2021| 10|  18|             40766|              359150|
|2021| 12|  55|             31592|              503289|
|2021|  1|  36|             17499|              123225|
|2021|  3|   1|             35998|              254990|
|2021|  6|  22|             53669|              656063|
|2021|  4|  77|              8059|              148052|
|2021|  1|  90|              1203|               23363|
|2021| 10|  41|             39283|              497941|
|2021| 10|  36|             40435|              475086|
|2021| 10|  64|             27652|              567812|
|2021| 10|  61|             31015|              

In [13]:
#se escribe el parquet tabla 10
result_df.write.parquet("hdfs:/analyticsdata/10.Tabla_anio_mes_edad_2021.parquet") 

In [14]:
spark.stop()